In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [19]:
def extract(w):
    i = -1
    for idx, c in enumerate(w):
        if c == "/":
            i = idx
    return [w[:i], w[i+1:]]

In [33]:
file = open('Brown_train.txt')
emissions = defaultdict(int)
tag_counts = defaultdict(int)
trigram_counts = defaultdict(int)
bigram_counts = defaultdict(int)
s_tags = set()
X = []
y = []

for l in file:
    word_tags = l.split()
    word_tags = ["*/*", "*/*"] + word_tags
    tags = []
    s = []
    
    for w in word_tags:
        try:
            w, t = w.split("/")
        except:
            w, t = extract(w)      
        emissions[w.lower()+t] += 1
        tag_counts[t] += 1
        tags.append(t)
        s_tags.add(t)
        s.append(w)
        
    y.append(tags)
    X.append(s)
    
    for i, t in enumerate(tags):
        if i+2 < len(tags):
            trigram_counts[tags[i]+tags[i+1]+tags[i+2]] += 1
        if i+1 < len(tags):
            bigram_counts[tags[i]+tags[i+1]] += 1

In [35]:
tags = s_tags
vocab_size = len(tags)
pi = defaultdict(lambda: defaultdict(int))
ws = defaultdict(lambda: defaultdict(lambda: ""))
pi[0]["**"] = 1
lam = 0.2

In [36]:
def q(v, w, u):
    try: 
        ans = (trigram_counts[w+u+v] + lam) / (bigram_counts[w+u] + lam * vocab_size)
    except:
        ans = 0
    return ans

In [37]:
def e(xk, v):
    return (emissions[xk.lower()+v] + lam) / (tag_counts[v] + lam * vocab_size)

In [38]:
def viterbi(sentence):
    n = len(sentence)
    max_pair = [None, 0]
    
    for k in range(1,n+1):
        for u in tags:
            for v in tags:
                max_w = None
                temp = 0
                for w in tags:
                    temp = pi[k-1][w+u] * q(v, w, u) * e(sentence[k-1], v)
                    if pi[k][u+v] < temp or max_w is None:
                        pi[k][u+v] = temp
                        max_w = w
                        
                ws[k][u+v] = max_w
                
        if k == n:
            if max_pair[0] is None or max_pair[1] < pi[n][u+v] * q(".", u, v):
                max_pair = [[u, v], pi[n][u+v] * q(".", u, v)]
            
            
    print(max_pair[0])
    answer = [*max_pair[0]]
    answer.reverse()
    for k in range(n-1, 0, -1):
        t = ws[k][answer[-1]+answer[-2]]
        answer.append(t)
  
    answer.reverse()
    return answer

In [39]:
l = viterbi(X[0])

['*', '*']


In [40]:
l, y[0]

(['*',
  '*',
  'X',
  '*',
  'ADP',
  'DET',
  'NOUN',
  'X',
  'X',
  'X',
  'X',
  'CONJ',
  'X',
  'X',
  'PRT',
  'X',
  '*',
  '*'],
 ['*',
  '*',
  'ADP',
  'DET',
  'NOUN',
  'NOUN',
  'NOUN',
  'VERB',
  'ADJ',
  'CONJ',
  'ADJ',
  'NOUN',
  'PRT',
  'VERB',
  'DET',
  'NOUN',
  '.'])

In [41]:
print(" ".join(X[0]))
print(" ".join(y[0]))
print(" ".join(l))

* * At that time highway engineers traveled rough and dirty roads to accomplish their duties .
* * ADP DET NOUN NOUN NOUN VERB ADJ CONJ ADJ NOUN PRT VERB DET NOUN .
* * X * ADP DET NOUN X X X X CONJ X X PRT X * *
